In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.nn.utils import parameters_to_vector
import torchvision.transforms as transforms

/home/ba13/anaconda3/envs/conda-torch/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/ba13/anaconda3/envs/conda-torch/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from model import Net
from hyperparams import _n_params_subnet, _base_path

In [3]:
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

Using device cuda:0


In [4]:
# Load the MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=f'{_base_path}/data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root=f'{_base_path}/data', train=False, download=True, transform=transforms.ToTensor())

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [5]:
net = Net().to(device)
net.load_state_dict(torch.load(f'{_base_path}/mnist_net.pth'))

<All keys matched successfully>

In [6]:
subnet_mask_indices = torch.load(f'{_base_path}/subnet_mask_indices.pt').to(device)

In [7]:
params_dict = {k: v for k, v in net.named_parameters() if v.requires_grad}
buffers_dict = {k: v for k, v in net.named_buffers()}

In [8]:
params_dict.keys(), buffers_dict.keys()

(dict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias']),
 dict_keys([]))

In [9]:
net = net.to(device)
params_dict = {key: value.to(device) for key, value in params_dict.items()}
buffers_dict = {key: value.to(device) for key, value in buffers_dict.items()}

n_params = len(parameters_to_vector(net.parameters()).detach())
print("n_params:", n_params)

H = torch.zeros(_n_params_subnet, _n_params_subnet, device=device)
loss = 0
n_data = 0
net.eval()
mean = parameters_to_vector(net.parameters())
X, _ = next(iter(train_loader))
with torch.no_grad():
    try:
        out = net(X[:1].to(device))
    except (TypeError, AttributeError):
        out = net(X.to(device))
n_outputs = out.shape[-1]
print("n_outputs:", n_outputs)
setattr(net, 'output_size', n_outputs)

n_params: 214538
n_outputs: 10


In [10]:
def model_fn_params_only(params_dict, buffers_dict):
    out = torch.func.functional_call(net, (params_dict, buffers_dict), X)
    return out, out

In [11]:
lossfunc = nn.CrossEntropyLoss(reduction='sum')
count = 0

In [12]:
X, y = next(iter(train_loader))
X, y = X.to(device), y.to(device)
print("Label:", y)
count += 1

Label: tensor([6], device='cuda:0')


In [13]:
net.zero_grad()
Js, f = torch.func.jacrev(model_fn_params_only, has_aux=True)(params_dict, buffers_dict)

Js = [
    j.flatten(start_dim=-p.dim())
    for j, p in zip(Js.values(), params_dict.values())
]
Js = torch.cat(Js, dim=-1)

Js = Js[:, :, subnet_mask_indices]
print("Sum:", Js.sum())
print(Js)
print("Count:", count)

Sum: tensor(0., device='cuda:0', grad_fn=<SumBackward0>)
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0',
       grad_fn=<IndexBackward0>)
Count: 1


In [ ]:
N = len(train_loader.dataset)
i = 0
for X, y in train_loader:
# X, y = next(iter(train_loader))
    i = i + 1
    if i % 100 == 0:
        print(i)
    net.zero_grad()
    X, y = X.to(device), y.to(device)
    
    with torch.no_grad():
        Js, f = torch.func.jacrev(model_fn_params_only, has_aux=True)(params_dict, buffers_dict)

    Js = [
        j.flatten(start_dim=-p.dim())
        for j, p in zip(Js.values(), params_dict.values())
    ]
    Js = torch.cat(Js, dim=-1)

    Js = Js[:, :, subnet_mask_indices]
    print(Js.sum(), Js.shape)


    ps = torch.softmax(f, dim=-1)
    H_lik = torch.diag_embed(ps) - torch.einsum('mk,mc->mck', ps, ps)
    H_batch = torch.einsum('bcp,bck,bkq->pq', Js, H_lik, Js)
    loss_batch = 1. * lossfunc(f, y)
    loss += loss_batch
    H += H_batch
    del Js, f, H_lik, H_batch, ps, loss_batch, X, y
n_data += N

In [ ]:
from torch.distributions.multivariate_normal import _precision_to_scale_tril

In [ ]:
sigma_noise = 1.
temperature = 1
sigma_noise = torch.tensor(sigma_noise, device=device)
sigma2 = sigma_noise.square()
_H_factor = 1 / sigma2 / temperature

In [ ]:
prior_precision = 1.
prior_precision_diag = torch.ones(_n_params_subnet, device=device)
posterior_precision = _H_factor * H + torch.diag(prior_precision_diag)
# posterior_precision = torch.diag(prior_precision_diag)
invsqrt_precision = _precision_to_scale_tril
posterior_scale = invsqrt_precision(posterior_precision)
scale = posterior_scale
posterior_covariance = scale @ scale.T

In [ ]:
posterior_covariance.shape